In [1]:
!pip install pydantic

### model_validator: wrap, before, after Modes
---

```
wrap [
    raw input
    ↓
    before
    ↓
    字段校验 + 字段类型转换
    ↓
    after
    ↓
    返回模型实例
]
```

`@model_validator(mode="before")` 常用于填补默认值和兼容旧的数据结构

In [1]:
from pydantic import BaseModel, model_validator
class A(BaseModel):
    id: str
    name: str

    @model_validator(mode="before")
    @classmethod
    def before_model_validate(cls, data):
        if not isinstance(data, dict):
            return data
        if "id" not in data:
            data["id"] = f"__{data["name"]}__"
        return data
A.model_validate({"name": "abc"})

A(id='__abc__', name='abc')

`@model_validator(mode="before")` 常用于字段校验和业务计算等后处理常见

In [2]:
from pydantic import BaseModel, model_validator
class A(BaseModel):
    price: float
    count: int
    total: float

    @model_validator(mode="after")
    def check_total(self):
        if self.price * self.count != self.total:
            raise ValueError("total 不正确")
        return self
A(price=12.4, count=2, total=24.8)

A(price=12.4, count=2, total=24.8)

### Union 分发机制
---

In [7]:
from typing import Literal, Union
from pydantic import BaseModel, TypeAdapter
class B(BaseModel):
    type: Literal["B"] = "B"
class C(BaseModel):
    type: Literal["C"] = "C"
TypeAdapter(Union[B, C]).validate_python({"type": "C"})

C(type='C')

核心流程是 Python Type → Json Schema → Rust Validator

对 `Union[B, C]`，生成的 Schema 类似：
```json
{
    "type": "union",
    "choices": [
        {"type": "model", "cls": B, "schema": {"type": "literal(\"B\")"}},
        {"type": "model", "cls": C, "schema": {"type": "literal(\"C\")"}},
    ],
    "mode": "smart"
}
```

由于示例中的模型都有 `Literal` 约束，Pydantic 会自动推导 discriminator 并构建 Tagged Union，在 Rust 层用 O(1) 哈希直接路由到 C 的校验器。

内部实质上是首先尝试 literal 校验，校验失败直接抛弃，然后对所有候选项打分，最终返回得分最高的模型实例。

In [8]:
class C1(C):
    name: str
TypeAdapter(Union[B, C, C1]).validate_python({"type": "C", "name": "abc"})

C1(type='C', name='abc')

### 序列化 Protocol
---

In [9]:
from typing import Protocol
from pydantic import BaseModel
class MyProto(Protocol): ...
class A(BaseModel):
    p: MyProto

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class '__main__.MyProto'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.12/u/schema-for-unknown-type

#### 解决方案
---

**解决方案一**

设置 `arbitrary_types_allowed=True`，并且将 Protocol 设置 `runtime_checkable` 使其可以在运行时被检查

这是最简单直接的方法。它告诉 Pydantic 允许任意类型，对于 Pydantic 无法识别的类型，它会直接存储 Python 对象本身，不进行额外的验证或序列化。

In [1]:
from typing import Protocol, runtime_checkable
from pydantic import BaseModel, ConfigDict

@runtime_checkable
class MyProto(Protocol):
    def say_hello(self) -> str: ...

class A(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    p: MyProto

class ConcreteImpl:
    def say_hello(self) -> str:
        return "Hello from ConcreteImpl!"

A(p=ConcreteImpl())

A(p=<__main__.ConcreteImpl object at 0x7912456c8e30>)

**解决方案二**

实现 `BaseModel.__get_pydantic_core_schema__` 函数。

对于 Protocol 本身，直接实现 `__get_pydantic_core_schema__` 并不常见，因为 Protocol 旨在定义接口而不是具体的实现或序列化逻辑。


以下是一个概念性的示例，展示如何为"所有符合 MyProto 的类型"提供一个简单的 Schema。实际应用中，你可能需要更复杂的逻辑来处理 Protocol 的具体实现。

In [2]:
import inspect
from typing import Protocol, TypeVar, runtime_checkable
from pydantic import BaseModel, ConfigDict
from pydantic_core import CoreSchema, core_schema

class MyProto(Protocol):
    def say_hello(self) -> str: ...

    @classmethod
    def __get_pydantic_core_schema__(cls, source, handler) -> CoreSchema:
        def validate_my_proto(value: object) -> MyProto:
            if not isinstance(value, object):
                raise TypeError("Expected an object.")
            if not hasattr(value, "say_hello") or not callable(getattr(value, "say_hello")):
                raise ValueError("Object must have a 'say_hello' method.")
            return value

        return core_schema.json_or_python_schema(
            python_schema=core_schema.no_info_after_validator_function(
                validate_my_proto, core_schema.any_schema()
            ),
            json_schema=core_schema.any_schema(),
            serialization=core_schema.plain_serializer_function_ser_schema(lambda v: v),
        )
class B(BaseModel):
    p: MyProto
class ConcreteImpl:
    def say_hello(self) -> str:
        return "Hello from ConcreteImpl!"
B(p=ConcreteImpl())

B(p=<__main__.ConcreteImpl object at 0x78d11873aa80>)